In [101]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [102]:
# Load environment variables
load_dotenv()

True

In [103]:
# DB connection
engine = create_engine(
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

In [104]:
# Helper Functions
def execute_query(query, params=None):
    with engine.connect() as conn:
        return pd.read_sql(query, conn, params=params)

def execute_update(query, params=None):
    with engine.begin() as conn:
        result = conn.execute(query, params)
        return result.rowcount

In [105]:
# Example 1: Batch Insert Single Record (CREATE Operations)
insert_query_1 = """
INSERT INTO earthquakes_batch (id, magnitude, place, time, longitude, latitude, depth, depth_category)
VALUES ('us7000l9q2', 4.5, 'Northern Sumatra, Indonesia',
        '2024-05-14 10:00:00+07', 96.789, 3.456, 65.4, 'Shallow')
RETURNING *;
"""

new_record = execute_query(insert_query_1)
print("Inserted Record:")
display(new_record)

Inserted Record:


,id,magnitude,place,time,longitude,latitude,depth,depth_category
0,us7000l9q2,4.5,"Northern Sumatra, Indonesia",2024-05-14 03:00:00+00:00,96.789,3.456,65.4,Shallow


In [106]:
# Example 2: Batch Insert (CREATE Operations)
batch_insert_query = """
INSERT INTO earthquakes_batch (id, magnitude, place, time, longitude, latitude, depth, depth_category)
VALUES
    ('us7000l9q3', 5.2, 'Banda Sea', '2024-05-14 11:00:00+07', 129.567, -4.123, 150.2, 'Intermediate'),
    ('us7000l9q4', 6.1, 'Java Sea', '2024-05-14 12:00:00+07', 106.789, -5.678, 320.5, 'Deep')
RETURNING *;
"""

batch_records = execute_query(batch_insert_query)
print("Batch Inserted Records:")
display(batch_records)

Batch Inserted Records:


,id,magnitude,place,time,longitude,latitude,depth,depth_category
0,us7000l9q3,5.2,Banda Sea,2024-05-14 04:00:00+00:00,129.567,-4.123,150.2,Intermediate
1,us7000l9q4,6.1,Java Sea,2024-05-14 05:00:00+00:00,106.789,-5.678,320.5,Deep


In [107]:
# Example 3: Conditional Insert (CREATE Operations)
insert_query_3 = """
WITH check_exists AS (
    SELECT id FROM earthquakes_batch
    WHERE latitude BETWEEN -6 AND 6
    AND longitude BETWEEN 110 AND 120
    AND time >= NOW() - INTERVAL '24 hours'
)
INSERT INTO earthquakes_batch (id, magnitude, place, time, longitude, latitude, depth, depth_category)
SELECT
    'us7000l9q5',
    4.8,
    'Sulawesi Sea',
    '2024-05-14 13:00:00+07',
    115.234,
    2.345,
    85.7,
    'Intermediate'
WHERE NOT EXISTS (SELECT 1 FROM check_exists)
RETURNING *;
"""

conditional_insert = execute_query(insert_query_3)
print("Conditional Insert Result:")
display(conditional_insert)

Conditional Insert Result:


,id,magnitude,place,time,longitude,latitude,depth,depth_category
0,us7000l9q5,4.8,Sulawesi Sea,2024-05-14 06:00:00+00:00,115.234,2.345,85.7,Intermediate


In [108]:
# Example 1: Select by Magnitude Range (READ Operations)
read_query_1 = """
SELECT *
FROM earthquakes_batch
WHERE magnitude BETWEEN 4.0 AND 5.0
ORDER BY time DESC
LIMIT 5;
"""

magnitude_range_results = execute_query(read_query_1)
print("Earthquakes with magnitude 4.0-5.0:")
display(magnitude_range_results)

Earthquakes with magnitude 4.0-5.0:


,id,magnitude,place,time,longitude,latitude,depth,depth_category
0,us7000pz3s,4.8,"63 km S of Bengkulu, Indonesia",2025-05-14 10:16:58+00:00,102.3665,-4.3636,63.893,Shallow
1,us7000pyz0,4.4,"28 km SE of Caburan, Philippines",2025-05-13 19:30:23+00:00,125.8215,5.7336,143.696,Intermediate
2,us7000pyy2,4.4,"173 km W of Gorontalo, Indonesia",2025-05-13 12:41:10+00:00,121.5159,0.6925,100.046,Intermediate
3,us7000pyus,4.1,"248 km ENE of Lospalos, East Timor",2025-05-12 23:23:59+00:00,128.9557,-7.4163,133.637,Intermediate
4,us7000pyul,4.5,"51 km WSW of Sungai Penuh, Indonesia",2025-05-12 22:40:06+00:00,100.9618,-2.2280,69.192,Shallow


In [109]:
# Example 2: Geographic Query (READ Operations)
read_query_2 = """
SELECT *
FROM earthquakes_batch
WHERE longitude BETWEEN 100 AND 110
  AND latitude BETWEEN -5 AND 5
  AND depth_category = 'Shallow'
ORDER BY time DESC
LIMIT 5;
"""

geographic_results = execute_query(read_query_2)
print("Shallow earthquakes in specified region:")
display(geographic_results)

Shallow earthquakes in specified region:


,id,magnitude,place,time,longitude,latitude,depth,depth_category
0,us7000pz3s,4.8,"63 km S of Bengkulu, Indonesia",2025-05-14 10:16:58+00:00,102.3665,-4.3636,63.893,Shallow
1,us7000pyul,4.5,"51 km WSW of Sungai Penuh, Indonesia",2025-05-12 22:40:06+00:00,100.9618,-2.2280,69.192,Shallow


In [110]:
# Example 3: Statistical Analysis (READ Operations)
read_query_3 = """
SELECT
    depth_category,
    COUNT(*) as event_count,
    ROUND(AVG(magnitude)::numeric, 2) as avg_magnitude,
    ROUND(MAX(magnitude)::numeric, 2) as max_magnitude,
    ROUND(MIN(depth)::numeric, 2) as min_depth,
    ROUND(MAX(depth)::numeric, 2) as max_depth
FROM earthquakes_batch
WHERE time >= NOW() - INTERVAL '30 days'
GROUP BY depth_category
ORDER BY event_count DESC;
"""

stats_results = execute_query(read_query_3)
print("Statistical Analysis by Depth Category:")
display(stats_results)

Statistical Analysis by Depth Category:


,depth_category,event_count,avg_magnitude,max_magnitude,min_depth,max_depth
0,Very Shallow,60,4.67,5.9,8.18,57.76
1,Intermediate,42,4.51,6.2,70.18,299.21
2,Deep,9,4.32,4.6,314.46,557.33
3,Shallow,6,4.68,4.8,63.89,69.45


In [120]:
# Example 1: Update Depth Categories (Update Operations)
from sqlalchemy import text

update_query_1 = text("""
UPDATE earthquakes_batch
SET depth_category =
    CASE
        WHEN depth <= 60 THEN 'Very Shallow'
        WHEN depth <= 70 THEN 'Shallow'
        WHEN depth <= 300 THEN 'Intermediate'
        ELSE 'Deep'
    END
WHERE depth <= 60 AND depth_category = 'Shallow';
""")

rows_updated = execute_update(update_query_1)
print(f"Number of depth categories updated: {rows_updated}")

# Check updated records
select_query_1 = text("""
SELECT *
FROM earthquakes_batch
WHERE depth_category = 'Very Shallow'
ORDER BY depth;
""")

updated_categories = execute_query(select_query_1)
display(updated_categories)

Number of depth categories updated: 0


,id,magnitude,place,time,longitude,latitude,depth,depth_category
0,us7000pw9q,4.8,"287 km N of Tobelo, Indonesia",2025-05-01 06:38:26+00:00,128.4159,4.2964,8.179,Very Shallow
1,us6000q6l0,4.5,"10 km SE of Labuha, Indonesia",2025-04-17 01:57:51+00:00,127.5400,-0.6919,10.000,Very Shallow
2,us7000py5d,4.5,"58 km NNE of Bima, Indonesia",2025-05-09 11:02:37+00:00,118.9414,-7.9715,10.000,Very Shallow
3,us7000px5i,4.4,"137 km SE of Sanana, Indonesia",2025-05-05 05:39:40+00:00,126.7938,-2.9968,10.000,Very Shallow
4,us7000pwyv,4.5,"63 km NNE of Pototano, Indonesia",2025-05-04 04:26:26+00:00,117.7102,-7.8887,10.000,Very Shallow
5,us7000pwu6,4.8,"234 km WSW of Bandar Lampung, Indonesia",2025-05-03 11:45:55+00:00,103.3802,-6.4072,10.000,Very Shallow
6,us7000pwj0,4.6,"14 km SSE of Bukittinggi, Indonesia",2025-05-02 14:07:44+00:00,100.4292,-0.4212,10.000,Very Shallow
7,us7000pw3z,4.4,"110 km ESE of Sarangani, Philippines",2025-04-30 22:53:49+00:00,126.2903,4.8505,10.000,Very Shallow
8,us7000puz1,4.4,"185 km NNE of Palu, Indonesia",2025-04-25 13:09:32+00:00,120.5665,0.6122,10.000,Very Shallow
9,us7000pu61,4.9,"157 km WSW of Fakfak, Indonesia",2025-04-22 17:15:54+00:00,130.9939,-3.4732,10.000,Very Shallow


In [112]:
# Example 2: Update Place Names (Update Operations)
from sqlalchemy import text

update_query_2 = text("""
UPDATE earthquakes_batch
SET place = REPLACE(place, 'Lospalos, Timor Leste', 'Lospalos, East Timor')
WHERE place LIKE '%Lospalos, Timor Leste%';
""")

rows_updated = execute_update(update_query_2)
print(f"Number of place names updated: {rows_updated}")

# Check updated records
select_query_2 = text("""
SELECT *
FROM earthquakes_batch
WHERE place LIKE '%Lospalos, East Timor%';
""")

updated_places = execute_query(select_query_2)
display(updated_places)

Number of place names updated: 0


,id,magnitude,place,time,longitude,latitude,depth,depth_category
0,us7000pyus,4.1,"248 km ENE of Lospalos, East Timor",2025-05-12 23:23:59+00:00,128.9557,-7.4163,133.637,Intermediate
1,us7000pyfm,4.2,"272 km NNE of Lospalos, East Timor",2025-05-10 23:24:08+00:00,128.1220,-6.3240,347.013,Deep
2,us7000pyc9,5.3,"282 km ENE of Lospalos, East Timor",2025-05-10 05:45:44+00:00,129.1380,-7.1234,164.942,Intermediate
3,us7000pwzj,4.4,"215 km NE of Lospalos, East Timor",2025-05-04 08:05:01+00:00,128.5350,-7.3244,160.170,Intermediate
4,us7000pw1n,4.6,"226 km ENE of Lospalos, East Timor",2025-04-30 15:43:29+00:00,128.7458,-7.4526,127.875,Intermediate
5,us7000pubs,4.4,"250 km ENE of Lospalos, East Timor",2025-04-23 02:10:39+00:00,128.9301,-7.3274,116.755,Intermediate
6,us6000q71f,4.1,"201 km NE of Lospalos, East Timor",2025-04-18 22:39:22+00:00,128.4584,-7.4213,167.023,Intermediate
7,us6000q6a4,4.1,"258 km NE of Lospalos, East Timor",2025-04-16 01:52:30+00:00,128.3910,-6.6448,280.800,Intermediate
8,us6000q6pn,4.8,"28 km SW of Lospalos, East Timor",2025-04-17 14:26:40+00:00,126.8377,-8.7277,22.377,Very Shallow


In [113]:
# Example 3: Update Depths for Biak Region (Update Operations)
update_query_3 = text("""
UPDATE earthquakes_batch
SET depth = round(depth::numeric, 2)
WHERE place LIKE '%Biak, Indonesia%'
AND depth < 30;
""")

rows_updated = execute_update(update_query_3)
print(f"Number of Biak records updated: {rows_updated}")

# Check updated records
select_query_3 = text("""
SELECT *
FROM earthquakes_batch
WHERE place LIKE '%Biak, Indonesia%'
ORDER BY depth;
""")

updated_depths = execute_query(select_query_3)
display(updated_depths)

Number of Biak records updated: 2


,id,magnitude,place,time,longitude,latitude,depth,depth_category
0,us7000pxru,4.9,"72 km SSE of Biak, Indonesia",2025-05-08 00:49:33+00:00,136.3046,-1.7928,23.93,Very Shallow
1,us7000pxht,4.6,"75 km SSE of Biak, Indonesia",2025-05-06 21:20:11+00:00,136.3595,-1.7991,27.35,Very Shallow


In [114]:
# Example 1: Delete Duplicates (Delete Operations)
delete_query_1 = """
WITH DuplicateRows AS (
    SELECT id,
           ROW_NUMBER() OVER (PARTITION BY longitude, latitude, time ORDER BY id) as row_num
    FROM earthquakes_batch
)
DELETE FROM earthquakes_batch
WHERE id IN (
    SELECT id
    FROM DuplicateRows
    WHERE row_num > 1
)
RETURNING *;
"""

deleted_duplicates = execute_query(delete_query_1)
print("Deleted duplicate records:")
display(deleted_duplicates)

Deleted duplicate records:


,id,magnitude,place,time,longitude,latitude,depth,depth_category


In [115]:
# Example 2: Delete by Criteria (Delete Operations)
delete_query_2 = """
DELETE FROM earthquakes_batch
WHERE magnitude < 3.0
  AND time < NOW() - INTERVAL '30 days'
RETURNING *;
"""

deleted_old_records = execute_query(delete_query_2)
print("Deleted old, low-magnitude records:")
display(deleted_old_records)

Deleted old, low-magnitude records:


,id,magnitude,place,time,longitude,latitude,depth,depth_category


In [116]:
# Example 3: Delete Invalid Coordinates (Delete Operations)
delete_query_3 = """
WITH invalid_records AS (
    DELETE FROM earthquakes_batch
    WHERE (latitude < -90 OR latitude > 90)
    OR (longitude < -180 OR longitude > 180)
    OR magnitude < 0
    OR depth < 0
    RETURNING *
)
SELECT * FROM invalid_records;
"""

deleted_invalid = execute_query(delete_query_3)
print("Deleted records with invalid coordinates:")
display(deleted_invalid)

Deleted records with invalid coordinates:


,id,magnitude,place,time,longitude,latitude,depth,depth_category
